# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisé en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par RFE (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_ess = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Regression SVM
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XG Boost
- Perceptron

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

In [5]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score

def compare_models_old(X, y):
    """ Fonction to compare the RMSE get with the
    most common Machine Learning Regressors.
    -----------
    Parameters:
    X: Array
        the array object holding data
    y: Array
        the target
    -----------
    Return:
        DataFrame
    """
    # Create models
    ridge = Ridge()
    lasso = Lasso()
    elastic_net = ElasticNet()
    svm_reg = LinearSVR()
    tree_reg = DecisionTreeRegressor()
    forest_reg = RandomForestRegressor()
    gbrt = GradientBoostingRegressor()
    xgbr = XGBRegressor()
    perceptron = Perceptron()
    models = [ridge,
              lasso,
              elastic_net,
              #svm_reg,
              tree_reg,
              forest_reg,
              gbrt,
              xgbr]
    # Train models
    scores = []
    names = []
    std_rmse = []
    for m in models:
        m.fit(X, y)
        m_scores = cross_val_score(m,X, y,
                                   scoring="neg_mean_squared_error",
                                   cv=10)
        m_scores = np.sqrt(-m_scores)
        m_names = type(m).__name__
        scores.append(m_scores.mean())
        std_rmse.append(m_scores.std())
        names.append(m_names)
    # Create the DataFrame
    df = pd.DataFrame({'RMSE_mean': scores, 'RMSE_std': std_rmse}, index=names)
    return df

In [6]:
df_comp =  compare_models_old(X, y)
df_comp

,RMSE_mean,RMSE_std
Ridge,0.976681,0.478513
Lasso,1.315544,0.171342
ElasticNet,1.285348,0.177976
DecisionTreeRegressor,0.431940,0.090451
RandomForestRegressor,0.282644,0.084923
GradientBoostingRegressor,0.208709,0.059106
XGBRegressor,0.217740,0.068303


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from functions.ml_modeling import linreg_bparams, compare_models

# Create models
ridge = linreg_bparams(X, y, Ridge())
lasso = linreg_bparams(X, y, Lasso())
elastic_net = linreg_bparams(X, y, ElasticNet(), elastic_net=True)
models = [ridge,
          lasso,
          elastic_net]

# Compare models
df = compare_models(X, y, models)
df

,RMSE_mean,RMSE_std
Ridge,0.900369,0.151509
Lasso,0.932346,0.327280
ElasticNet,0.905065,0.195328


In [ ]:
ridge1

In [11]:
from sklearn.linear_model import RidgeCV

ridge1 = linreg_bparams(X, y, Ridge())
print(ridge1.get_params())
ridge2 = RidgeCV().fit(X, y)
print(ridge2.alpha_)

{'alpha': 501.18723362727246, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}
10.0


In [13]:
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

ridge = Pipeline([('rfe', RFECV(Ridge(), scoring="neg_mean_squared_error")),
                  ('model', Ridge())])
models = [ridge]

# Compare models
df = compare_models(X, y, models)
df

,RMSE_mean,RMSE_std
Pipeline,0.979449,0.456546


In [28]:
rfe = RFECV(Ridge(), scoring="neg_mean_squared_error")
rfe.fit(X, y)
print(rfe.n_features_)
test = rfe.grid_scores_
n = np.argmax(test)+1
n

20


20

In [31]:
test = rfe.grid_scores_
test

array([-1.86972323, -1.650277  , -1.51539961, -1.4553617 , -1.40025137,
       -1.26852857, -1.2427139 , -1.20512369, -1.16385711, -1.14540183,
       -1.14294312, -1.12893217, -1.12633595, -1.10722573, -1.10107955,
       -1.09696905, -1.08143608, -1.08142703, -1.0696636 , -1.06436046,
       -1.07713313, -1.07957808, -1.19321447, -1.19409086, -1.2018119 ,
       -1.19545032, -1.1995483 , -1.20827348, -1.21032119, -1.22448474,
       -1.22924284, -1.23596511, -1.23787362, -1.23578763, -1.2370925 ,
       -1.24903125, -1.22887291, -1.23715958, -1.2372978 , -1.2362568 ,
       -1.23653754, -1.23636809, -1.23518641, -1.23520672, -1.23136341,
       -1.22760552, -1.22486223, -1.223326  , -1.22404046, -1.22403069,
       -1.22406142, -1.22404492, -1.22417888, -1.22418561, -1.2241863 ])

In [21]:
rfe = RFECV(Ridge())
rfe.fit(X, y)
print(rfe.n_features_)

20


In [16]:
X.shape[1]

55

In [17]:
cols = data.iloc[:, :-6].columns
cols[rfe.support_]

Index(['Latitude', 'GHGEmissionsIntensity', 'NumberofBuildings',
       'PropertyGFATotal', 'YearBuilt', 'PropertyGFABuilding(s)_ratio',
       'Self-Storage Facility', 'Warehouse', 'Distribution Center',
       'Worship Facility', 'DELRIDGE NEIGHBORHOODS', 'DOWNTOWN', 'Delridge',
       'LAKE UNION', 'NORTHEAST', 'NORTHWEST', 'North', 'Northwest',
       'SOUTHEAST', 'SOUTHWEST'],
      dtype='object')

In [22]:
cols = data.iloc[:, :-6].columns
cols[rfe.support_]

Index(['Latitude', 'GHGEmissionsIntensity', 'NumberofBuildings',
       'PropertyGFATotal', 'YearBuilt', 'PropertyGFABuilding(s)_ratio',
       'Self-Storage Facility', 'Warehouse', 'Distribution Center',
       'Worship Facility', 'DELRIDGE NEIGHBORHOODS', 'DOWNTOWN', 'Delridge',
       'LAKE UNION', 'NORTHEAST', 'NORTHWEST', 'North', 'Northwest',
       'SOUTHEAST', 'SOUTHWEST'],
      dtype='object')

In [27]:
test = rfe.grid_scores_
n = np.argmax(test)+1
n

20

In [9]:
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

ridge = Pipeline([
    ('rfe', RFECV(Ridge())),
    ('model', FunctionTransformer(linreg_bparams,
                                  kw_args={'y': y,
                                           'estimator': Ridge()}))])
lasso = Pipeline([('rfe', RFECV(Lasso())),
                  ('model', linreg_bparams(X, y, Lasso()))])
elastic_net = Pipeline([('rfe', RFECV(ElasticNet())),
                  ('model', linreg_bparams(X, y, ElasticNet(), elastic_net=True))])
models = [ridge,
          lasso,
          elastic_net]

# Compare models
df = compare_models(X, y, models)
df

AttributeError: 'FunctionTransformer' object has no attribute 'predict'